In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from PIL import Image
import os
from pathlib import Path
from utils import *
from allResNets import *
from engine import *
from data_transforms import create_transform
from dataloaders import *

change_to_disk()
data_dir = Path("Quality/data/single_data/split_1k_single")
models_path = Path('Quality/Models_test/')
check_dir(models_path)

In [2]:
image_shape = (1080, 1920)
resize_factor = 0.3
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]
BATCH_SIZE = 16

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

train_dl, validation_dl, test_dl, train_data, validation_data, test_data = create_dataloaders_multilabel(data_dir, transform = data_transform, batch_size=BATCH_SIZE, num_workers= NUM_WORKERS)

In [3]:
NUM_EPOCHS = 100
lr = 0.001
device = get_device()

resnet18 = ResNet18()

#Combines the sigmoid with the BinaryCrossEntropy
loss_fn = nn.BCEWithLogitsLoss()

In [4]:
model_name = "ResNet18"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet18
resnet18.to(device)
nadam_optim = torch.optim.NAdam(params=resnet18.parameters(), lr=lr)
train_resnet18_results, train_time_resnet18 = train(resnet18, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

Quality\Models_test\ResNet18 does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5644 | train_acc: 0.4169 | validation_loss: 2.0659 | validation_acc: 0.3388
Epoch: 2 | train_loss: 0.4760 | train_acc: 0.5202 | validation_loss: 0.7925 | validation_acc: 0.4901
Epoch: 3 | train_loss: 0.4162 | train_acc: 0.6149 | validation_loss: 0.4373 | validation_acc: 0.6283
Epoch: 4 | train_loss: 0.3726 | train_acc: 0.6716 | validation_loss: 0.4564 | validation_acc: 0.5592
Epoch: 5 | train_loss: 0.3544 | train_acc: 0.6933 | validation_loss: 0.4681 | validation_acc: 0.5789
Epoch: 6 | train_loss: 0.3121 | train_acc: 0.7391 | validation_loss: 0.3232 | validation_acc: 0.7697
Epoch: 7 | train_loss: 0.2894 | train_acc: 0.7741 | validation_loss: 0.3034 | validation_acc: 0.7533
Epoch: 8 | train_loss: 0.2805 | train_acc: 0.7846 | validation_loss: 0.3015 | validation_acc: 0.7763
Epoch: 9 | train_loss: 0.2730 | train_acc: 0.7958 | validation_loss: 0.3534 | validation_acc: 0.7697
Epoch: 10 | train_loss: 0.2329 | train_acc: 0.8241 | validation_loss: 0.3589 | validation_a

In [5]:
eval_resnet18_results = eval_model(resnet18, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(16,3,resize[0],resize[1]).to(device))
resnet18_results = [train_resnet18_results, train_time_resnet18, eval_resnet18_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, resnet18, resnet18_results)

  0%|          | 0/19 [00:03<?, ?it/s]

In [6]:
resnet50 = ResNet50()
model_name = "ResNet50"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet50
resnet50.to(device)
nadam_optim = torch.optim.NAdam(params=resnet50.parameters(), lr=lr)
train_resnet50_results, train_time_resnet50 = train(resnet50, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

Quality\Models_test\ResNet50 does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6061 | train_acc: 0.3677 | validation_loss: 0.9642 | validation_acc: 0.3816
Epoch: 2 | train_loss: 0.4989 | train_acc: 0.4665 | validation_loss: 0.7760 | validation_acc: 0.4079
Epoch: 3 | train_loss: 0.4672 | train_acc: 0.5273 | validation_loss: 0.7949 | validation_acc: 0.4276
Epoch: 4 | train_loss: 0.4442 | train_acc: 0.5523 | validation_loss: 0.4605 | validation_acc: 0.4901
Epoch: 5 | train_loss: 0.4083 | train_acc: 0.5894 | validation_loss: 11.3036 | validation_acc: 0.4432
Epoch: 6 | train_loss: 0.4123 | train_acc: 0.5986 | validation_loss: 0.5460 | validation_acc: 0.4901
Epoch: 7 | train_loss: 0.3935 | train_acc: 0.6328 | validation_loss: 1.1681 | validation_acc: 0.4934
Epoch: 8 | train_loss: 0.3312 | train_acc: 0.7183 | validation_loss: 8.8096 | validation_acc: 0.4145
Epoch: 9 | train_loss: 0.3379 | train_acc: 0.7236 | validation_loss: 0.4141 | validation_acc: 0.6546
Epoch: 10 | train_loss: 0.3136 | train_acc: 0.7641 | validation_loss: 0.4500 | validation_

In [ ]:
eval_resnet50_results = eval_model(resnet50, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(16,3,resize[0],resize[1]).to(device))
resnet50_results = [train_resnet50_results, train_time_resnet50, eval_resnet50_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, resnet50, resnet50_results)

In [ ]:
from MobileNetV3 import *

mobnet_sml = MobileNetSmall()
model_name = "MobileNetSmall"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train MobileNetV3 Small
mobnet_sml.to(device)
nadam_optim = torch.optim.NAdam(params=mobnet_sml.parameters(), lr=lr)
train_mobnet_sml_results, train_time_mobnet_sml = train(mobnet_sml, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)